# Segmenting and Clustering Neighborhoods in Toronto

## Assignment Instructions:

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## Web Scraping:

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

rawdata = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(rawdata,'lxml')

data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

## Dataframe Creation:

In [ ]:

#convert list into Pandas DataFrame
toronto_df = pd.DataFrame(data = data,columns = columns)

toronto_df.head()


## Dataframe Formatting:

Remove Boroughs that are 'Not assigned'

In [ ]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()


For neighbourhoods that are "Not assigned", apply their Borough as their Neighbourhood

In [ ]:
toronto_df['Neighbourhood'].replace("Not assigned", toronto_df["Borough"],inplace=True)
toronto_df.head()


Group all neighborhoods with the same postal code

In [ ]:
toronto_df = toronto_df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
toronto_df.head()

## Final Dataframe Shape:

In [ ]:
toronto_df.head()

print("Shape:", toronto_df.shape)

---

## Retrieving Longitude and Latitude Coordinates:

In [ ]:
geo_df = pd.read_csv("https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv")
geo_df.head()

Merge Neigbourhood and Latitude/Logitude Data

In [ ]:
TorGeo_df = pd.merge(toronto_df, geo_df, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
TorGeo_df.head()

---

## Analyzing and Clustering Toronto Data:

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes 

In [ ]:
import folium
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 

In [ ]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto